In [1]:
from fastai.conv_learner import *
from fastai.dataset import *

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

import torch as th
th.cuda.set_device(1)

/home/kownse/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH = './'
TRAIN = '../input/train_v2/'
TEST = '../input/test_v2/'
SEGMENTATION = '../input/train_ship_segmentations_v2.csv.zip'
exclude_list = ['6384c3e78.jpg','13703f040.jpg', '14715c06d.jpg',  '33e0ff2d5.jpg',
                '4d4e09f2a.jpg', '877691df8.jpg', '8b909bb20.jpg', 'a8d99130e.jpg', 
                'ad55c3143.jpg', 'c8260c541.jpg', 'd6c7f17c7.jpg', 'dc3e7c901.jpg',
                'e44dffe88.jpg', 'ef87bad36.jpg', 'f083256d8.jpg'] #corrupted image

In [3]:
test_names = [f for f in os.listdir(TEST)]
for el in exclude_list:
    if(el in test_names): test_names.remove(el)

In [4]:
class pdFilesDataset(FilesDataset):
    def __init__(self, fnames, path, transform):
        self.segmentation_df = pd.read_csv(SEGMENTATION).set_index('ImageId')
        super().__init__(fnames, transform, path)
    
    def get_x(self, i):
        img = open_image(os.path.join(self.path, self.fnames[i]))
        if self.sz == 768: return img 
        else: return cv2.resize(img, (self.sz, self.sz))
    
    def get_y(self, i):
        if(self.path == TEST): return 0
        masks = self.segmentation_df.loc[self.fnames[i]]['EncodedPixels']
        if(type(masks) == float): return 0 #NAN - no ship 
        else: return 1
    
    def get_c(self): return 2 #number of classes
    
def get_data(sz,bs, tr_n, val_n):
    #data augmentation
    aug_tfms = [RandomRotate(20, tfm_y=TfmType.NO),
                RandomDihedral(tfm_y=TfmType.NO),
                RandomLighting(0.05, 0.05, tfm_y=TfmType.NO)]
    tfms = tfms_from_model(arch, sz, crop_type=CropType.NO, tfm_y=TfmType.NO, 
                aug_tfms=aug_tfms)
    ds = ImageData.get_ds(pdFilesDataset, (tr_n[:-(len(tr_n)%bs)],TRAIN), 
                (val_n,TRAIN), tfms, test=(test_names,TEST))
    md = ImageData(PATH, ds, bs, num_workers=nw, classes=None)
#     md.is_multi = False
    return md

In [5]:
nw = 4   #number of workers for data loader
arch = resnet34 #specify target architecture

In [6]:
folds = pd.read_csv('../input/folds.csv')

In [ ]:
for i in range(5):
    tr_n = folds.loc[folds.fold != i, 'ImageId'].values.tolist()
    val_n = folds.loc[folds.fold == i, 'ImageId'].values.tolist()
    
    for el in exclude_list:
        if(el in tr_n): tr_n.remove(el)
        if(el in val_n): val_n.remove(el)
    
    sz = 256 #image size
    bs = 64  #batch size

    md = get_data(sz,bs, tr_n, val_n)
    learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    
    learn.fit(2e-3, 1)
    
    learn.unfreeze()
    lr=np.array([1e-4,5e-4,2e-3])
    
    learn.fit(lr, 1, cycle_len=3, use_clr=(20,8))
    path_256 = 'Resnet34_lable_256_fold{}_1'.format(i)
    learn.save(path_256)
    
    
    sz = 384 #image size
    bs = 32  #batch size

    md = get_data(sz,bs, tr_n, val_n)
    learn = ConvLearner.pretrained(arch, md, ps=0.5) #dropout 50%
    learn.opt_fn = optim.Adam
    learn.unfreeze()
    lr=np.array([1e-4,5e-4,2e-3])
    learn.load(path_256)
    
    learn.fit(lr/2, 1, cycle_len=10, use_clr=(20,8)) #lr is smaller since bs is only 32
    learn.save('Resnet34_lable_384_fold{}_1'.format(i))
    
    
    log_preds,y = learn.predict_with_targs(is_test=True)
    probs = np.exp(log_preds)[:,1]
    
    df = pd.DataFrame({'id':test_names, 'p_ship':probs})
    df.to_csv('../result/ship_detection_fold{}.csv'.format(i), header=True, index=False)

/home/kownse/anaconda3/lib/python3.6/site-packages/fastai/initializers.py:6: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  if hasattr(m, 'weight'): init_fn(m.weight)


epoch      trn_loss   val_loss   accuracy                      
    0      0.187474   0.141278   0.947062  



epoch      trn_loss   val_loss   accuracy                      
    0      0.107781   0.082195   0.970507  
 71%|███████▏  | 1719/2406 [12:05<04:49,  2.37it/s, loss=0.0723]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



epoch      trn_loss   val_loss   accuracy                       
    0      0.096629   0.069145   0.975803  
 55%|█████▍    | 2628/4813 [19:26<16:09,  2.25it/s, loss=0.0916]